In [ ]:
from src import clear_subs

clear_subs()

In [2]:
from os import symlink
from typing import Any, Optional
from pathlib import Path
from json import dump, load, JSONEncoder
from numpy import ndarray
from pydantic import BaseModel


class JSONSerialize(JSONEncoder):
    """
    Handmade JSON encoder that correctly encodes special structures.
    """

    def default(self, obj: Any):
        if type(obj) is ndarray:
            return obj.tolist()
        elif isinstance(obj, BaseModel):
            return obj.__dict__
        else:
            JSONEncoder().default(obj)


def save_base_model(obj: Any, name: str, path: Path):
    """
    Saves a JSON serializable type.
    """
    path.mkdir(exist_ok=True, parents=True)
    with open(path.joinpath(name + ".json"), "w") as file:
        dump(obj, fp=file, cls=JSONSerialize)


def load_base_model(
    name: str,
    path: Path,
    base_model_type: Optional[Any] = None,
) -> Any:
    """
    Loads a JSON serializable type.
    """
    with open(path.joinpath(name + ".json"), "r") as file:
        loaded_content = load(fp=file)
        return loaded_content if not base_model_type else base_model_type(**loaded_content)

In [4]:
obj = {i: [(j, [k for k in range(10)])for j in range(10, 20)] for i in range(20, 30)}

In [11]:
save_base_model(obj=obj, name="true_file", path=Path("true_files/test"))

In [4]:
symlink(src=Path("true_file.json"), dst=Path("symlink_file.json"))

In [16]:
load_base_model(name="symlink_file", path=Path("true_files"))

{'20': [[10, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [11, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [12, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [13, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [14, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [15, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [16, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [17, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [18, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [19, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]],
 '21': [[10, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [11, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [12, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [13, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [14, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [15, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [16, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [17, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [18, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [19, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]],
 '22': [[10, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [11, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [12, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [13, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
  [14, [0, 1, 2, 3, 